# Resolução de Sistemas Não Lineares

Um sistema não linear é um conjunto de equações com múltiplas vaiáveis, onde pelo menos uma equação é não linear, ou seja, não pode ser descrito apenas com somas, subtrações e multiplicações por constantes. Não conseguimos usar métodos de resolução de sistemas lineares em sistemas não lineares. Podemos usar alguns métodos numéricos iterativos, destacando o método de Newton-Raphson

Aqui nós iremos fazer a implementação desse método para a resolução de um sistema não linear qualquer em Python. O algoritmo usará Sympy para simbolizar as equações, e numpy para fazer o tratamento com os números. Vamos precisar importar ambos para o nosso ambiente.

In [49]:
import numpy as np
import sympy as sp


## Método de Newton-Raphson

### Matriz Jacobiana
Antes de explicarmos como funciona o método de Newton-Raphson, vamos precisar explicar o que é uma matriz Jacobiana. Uma matriz Jacobiana é uma matriz com todas as derivadas parciais de um sistema de equação. Seja o sistema $F=\{f_1,f_2,\dots,f_n\}$, em que cada função recebe as variáveis $\{x_1,x_2,\dots,x_n\}$:

$$
J_F(x_1,\dots,x_n)=\frac{\delta( f_1,\dots, f_n)}{\delta(x_1,\dots,x_n)}=
\begin{bmatrix}

\frac{\delta f_1}{\delta x_1} & \dots & \frac{\delta f_1}{\delta x_n} \\

\vdots & \ddots & \vdots \\

\frac{\delta f_m}{\delta x_1} & \dots & \frac{\delta f_m}{\delta x_n}

\end{bmatrix}
$$

Abaixo nós temos uma implementação da obtenção da matriz Jacobiana. Temos duas funções, uma para obter todas as variáveis da função, e outra para obter a matriz Jacobiana em questão. A implementação da função para obter variáveis foi feita para simplificar a utilização do algoritmo para o usuário

In [50]:
'''
    Descobre todas as variáveis livres de um sistema 

    Args:
        F: O sistema, representado como uma lista com todas as funções no formato simbólico do Sympy
    Retorno:
        Lista com todas as variaveis do sistema
'''
def variaveis(F: list[sp.Expr]) -> list[sp.Symbol]:
    allvars = [f.free_symbols for f in F]  # Cria uma lista com todas as variáveis de cada função
    return list(set.union(*allvars)) # type: ignore

'''
    Retorna a Jacobiana de um sistema de funções.
    Args:
        F: O sistema, representado como uma lista com todas as funções no formato simbólico do Sympy
    Retorno:
        A matriz jacobiana, uma matriz com todas as derivadas parciais
'''
def jacobiana(F: list[sp.Expr]) -> list[list[sp.Expr]]:
    J = []
    derivadas = []

    allvars = [f.free_symbols for f in F]  # Cria uma lista com todas as variáveis de cada função
    vars = list(set.union(*allvars)) # Faz a união de todas as variáveis em uma lista

    for i in range(len(F)): # Começa a iterar pela lista de funções
        for j in vars: # Começa a iterar pela lista de variáveis
            derivadas.append(sp.diff(F[i], j)) # Adiciona numa lista a derivada parcial da função, para cada variável
        J.append(derivadas)
        derivadas = []

    return J # Transforma a matriz de derivadas parciais numa matriz do SymPy


### Método de Newton-Raphson

Agora o método em questão. Vamos precisar seguir um passo a passo, que será repetido a cada iteração.
Dados $x⁰$, $\epsilon_1$, $\epsilon_2 > 0$

1. Calcular $F(x^k)$ e $J(x^k)$
2. Se $\|F(x^k)\|_\infin<\epsilon_1$, então $x^*=x^k$ e pare; caso contrário, continue
3. Obtenha $S^k$, solução do sistema linear $J(x^k).S^k=-F(x^k)$
4. Faça: $x^{k+1}=x^k+S^k$
5. Se $\|x^{k+1}-x^k\|_\infin<\epsilon_2$, faça $x^*=x^{k+1}$ e pare; caso contrário, continue
6. $k=k+1$ e volte ao passo 1

Segue o algoritmo implementado em Python:

In [ ]:
'''
    Descobrir os zeros das funções em um sistema não linear, aplicando o método de Newton-Raphson

    Args:
        F: O sistema não linear, representado por uma lista com as funções em formato simbólico do Sympy 
        estimativa: Uma lista com as estimativas iniciais
        epsilon1: Primeiro critério de parada (Fxk < epsilon)
        epsilon2: Segundo critério de parada (xk1 - xk < epsilon)
        maximoIteracoes: Maximo de iterações desejada [opcional]
        verbose: Verdadeiro caso queira descrição maior de como foi o ciclo de iterações
    
    Retorno:
        Um array numpy com os zeros das funções no sistema não linear F
'''
def newtonRaphson(F: list[sp.Expr], estimativa: list[float], epsilon1 = 10**-3, epsilon2 = 10**-3, maximoIteracoes=500, verbose=False) -> np.typing.NDArray[np.float64]:
    if len(F) != len(estimativa): # Verifica se o sistema tem tamanho diferente da estimativa inicial
        print("Tamanho da estimativa é diferente do sistema!")
        raise ArithmeticError # Levanta erro aritmético
    
    vars = variaveis(F) # Pega todas as variáveis do sistema de funções

    if len(estimativa) != len(vars): # Verifica se a quantidade de variáveis no sistema bate com a quantidade de valores na estimativa
        print("Estimativa inicial tem mais variáveis do que o sistema permite")
        raise ArithmeticError
    
    Fxk = np.array([])
    Jxk = np.array([])
    xk = np.array([np.float64(i) for i in estimativa]) # Transcrevendo a estimativa inicial para xk, para simplificar a leitura do algoritmo

    J = jacobiana(F) # Calcula a jacobiana da função
    lF = [sp.lambdify(vars, f, "numpy") for f in F] # Transforma todos os símbolos em F para funções lambda
    lJ = [[sp.lambdify(vars, f, "numpy") for f in j] for j in J] # Transforma todos os símbolos em J para funções lambda

    # Começa as iterações
    for n in range(maximoIteracoes): # Critério de parada, para evitar infinitas iterações
        Fxk = np.array([f(*xk) for f in lF]) # Pegamos os resultados do sistema F
        Jxk = np.array([[f(*xk) for f in j] for j in lJ]) # Pegamos os resultados das funções da matriz Jacobiana

        if np.linalg.norm(Fxk, np.inf) < epsilon1: # Primeiro critério de parada, verifica se algum resultado Fxk é menor que o primeiro epsilon
            if verbose:
                print(f"Encontrado zero das funções no ciclo {n+1}: {xk}")
            return xk
        
        Sk = np.linalg.solve(Jxk, -Fxk) # Resolução do sistema linear
        xk1 = xk+Sk # Reescrevendo xk1
        if np.linalg.norm(xk1 - xk, np.inf) < epsilon2: # Segundo critério de parada, verifica se a diferença para o resultado anterior é menor que epsilon
            if verbose:
                print(f"Encontrado zero das funções no ciclo {n+1}: {xk}")
            return xk1
        
        if verbose:
            print(f"Matriz resposta no ciclo {n+1}: {xk}")
        xk = xk1 # Preparando para a próxima iteração

    print("Máximo de iterações ultrapassado!")
    if verbose:
        print("Matriz resposta encontrada: {xk}")
    return xk

Agora nós temos tudo necessário para aplicar o método. Vamos testar com o seguinte sistema:

$$
\begin{cases}

F_1(x_1,x_2)=\sin(x_1)+x_2^2-1=0 \\
F_2(x_1,x_2)=x_1^2+\cos(x_2)-2=0 \\

\end{cases}\\
x^0=\begin{pmatrix}
0,5\\0,5
\end{pmatrix} \\

\epsilon_1=\epsilon_2=10^{-3}

\\~\\\text{Resposta Esperada}\\~\\

x_4^*=\begin{pmatrix}
1,034\\0,375
\end{pmatrix}
$$

In [66]:
x1, x2 = sp.symbols('x1 x2')
print(newtonRaphson([sp.sin(x1)+x2**2-1, x1**2+sp.cos(x2)-2],[0.5,0.5], verbose=True))

Matriz resposta no ciclo 1: [0.5 0.5]
Matriz resposta no ciclo 2: [1.20536546 0.15155803]
Matriz resposta no ciclo 3: [1.04312522 0.48488618]
Matriz resposta no ciclo 4: [1.03433425 0.38726511]
Encontrado zero das funções no ciclo 5: [1.03415167 0.37512454]
[1.03415167 0.37512454]
